# 

last updated 11 03 24

This notebook is to get the run times for each model on the highets and lowest Resolutions; to estimate an average run time.IG DICITONARY!



#!pip install ipython

In [1]:
# 09 01 25
# running three epochs to collect current sample images - is the black line still there or fixed? - fnding out

In [2]:
# imports
import torch

import torch.nn as nn
#from torchvision.models import vgg16
#import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim
#from torchvision.models import vgg16
from torch.utils.data import DataLoader
#from torch.Utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

import numpy as np
import cv2


from datetime import date
from tqdm import tqdm
import pprint
import collections
from IPython.display import clear_output
import time
import random
import cv2

import csv
import json
import pickle
import os

import wandb

import sys
sys.path.append('../.')
from functions import import_imagedata, ImageProcessor, label_oh_tf, IDSWDataSetLoader2
from fns4wandb import set_lossfn
#from architectures import sevennet, smallnet1, smallnet2, smallnet3
from architectures import PrintLayer, sixnet
from loop_fns import loop#, loop_batch, test_loop_batch
from plotting import learning_curve, accuracy_curve, plot_confusion

from copy import deepcopy

#import torch.Utils.data.DataLoader as DataLoader

In [3]:
#p = torch.cuda.memory_summary(device, abbreviated=False)
#Pp = pprint.PrettyPrinter(indent=4)
#Pp.pprint(p)

In [4]:
# file paths
_save_location = r'/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/6c3l/' #vgg16

data_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'

gitHASH = 'f7ff2f83eded46c54339d744170788e366c797da'

In [5]:
wandb.login()


wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [6]:
#!pip install datetime

d = date.today()
#print(str(d), type(str(d)))

452 144 5/452 *100 = 1%
226 72 5/226 *100 = 2%
113 36 5/113 *100 = 4% -- 2/113 *100= 1.7% ~ 2%
57 18 (56.5,) 5/57 *100 = 8% -- 2/57 *100 = 3.5% ~ 4%. 1/57 = 1.75%
29 9 (28.5,) 5/29 *100 = 17% -- 2/29 *100 = 6.89 ~ 7% 1/28 = 3.57 ~ 4%
15 5 (14.5, 4.5)
8 3 (7.5,2.5)
4, 2 (, 1.5)

In [7]:
# dictionaries                                                                                  * * * *   SETTINGS   * * * *

date = date.today()

model_card_6c3l = {'name': '6c3l', 'model': '6c3l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[272384,    # 452 144 # p5 (32x272384 and 248832x100)
                            71680,      # 226 72 # p5(32x71680 and 59904x100)
                            16640,      # 113 36 # p2(32x16640 and 11264x100)
                            3840,       # 57 18 # p1(32x3840 and 1536x100)
                            1024,           # 29 9 (32x1024 and 172032x100) (32x193024 and 172032x100)
                            193024,          # 15 5  (256x1x0) (32x193024 and 1024x100)
                            193024,         # 8 3
                              ], 
                   'idx': 1,
                  'dropout':0.2}


resolution_card_452144 = {'resolution':[452,144], 'padding':5, 'index':0}
resolution_card_22672 = {'resolution':[226,72], 'padding':5, 'index':1}
resolution_card_11336 = {'resolution':[113,36], 'padding':2, 'index':2}
resolution_card_5715 = {'resolution':[57,18], 'padding':1, 'index':3}

resolution_card_299 = {'resolution':[29,9], 'padding':0, 'index':4} # 
resolution_card_155 = {'resolution':[15,5], 'padding':0, 'index':5}
resolution_card_83 = {'resolution':[8,3], 'padding':0, 'index':6}



resolution_cards = [resolution_card_452144,resolution_card_22672,resolution_card_11336,resolution_card_5715,resolution_card_299,resolution_card_155,resolution_card_83] # resolution_card_452144, resolution_card_22672, resolution_card_11336, resolution_card_5715,
                   # resolution_card_299, resolution_card_83
#resolution_cards = [resolution_card_11336]

#learning_rate_cards = [5e-5, 6e-5, 8e-5]
#learning_rate_cards = [8.21592E-05, 6.62E-05, 6.01E-05, 5.97E-05]
learning_rate_cards=  [1e-4]#[0.1,0.01, 1e-3,1e-5]#, 6e-5, 7e-5, 8e-5] 0.1,0.01, 1e-3, 1e-4, 1e-5
#wd_cards = [4e-5, 5e-5, 3.00E-05, 2.00E-05]
wd_cards =[0]
#scheduler_cards = [0]#, 0.1, 0.2]

seeds = [42]#,2,3] # 4, 5,6

#model_cards =[model_card_vgg, model_card_7c3l, model_card_4c3l, model_card_3c2l, model_card_2c2l]
model_cards =[model_card_6c3l]

loss_fn_cards = ['MSE'] #,'CrossEntropy' 
                        
config = dict({'parameters': 'parameters for big loop run'})
config.update({'model_cards':model_cards})
config.update({'resolution_cards':resolution_cards})
config.update({'learning_rate_cards':learning_rate_cards})
config.update({'wd_cards':wd_cards})
config.update({'seeds':seeds})
config.update({'loss_fn_cards': loss_fn_cards})


config.update({'batch_size': 64})
config.update({'epochs': 3})

#print(model_card_vgg)
#print('')
#Pp.pprint(Config) # dictionary of dictionaries of lists and lists of dictionaries

In [8]:
   

Pp = pprint.PrettyPrinter(indent=4)

def save2csv_nest_dict(nested_dict, file_name, save_location:str):
    # flattern nested dictionary
    flatterend_dict = {}
    for k,v in nested_dict.items():
        if isinstance(v, dict):
            for nested_key, nested_val in v.items():
                flatterend_dict[f"{k}_{nested_key}"] = nested_val
        else:
            flatterend_dict[k] =v
    
    columns = list(flatterend_dict.keys())
    
    with open(save_location+str(file_name)+'.csv', "a+", newline="") as f:
        # using dictwriter
        writer = csv.DictWriter(f, fieldnames=columns)
        # using writeheader function
        if f.tell() == 0:
            writer.writeheader()
        writer.writerow(flatterend_dict)
        f.close()

# check dictionary values for json and csv

def check_obj4np(obj):
    if isinstance(obj, dict):
        return {key: check_obj4np(value) for key, value in obj.items()}
    if isinstance(obj,list):
        return [check_obj4np(item) for item in obj]
    if isinstance(obj,np.ndarray):
        return obj.tolist()
    if isinstance(obj, torch.Tensor):
        return obj.tolist()
    else:
        return obj

# save to json
def save2josn_nested_dict(nested_dict, file_name, save_location:str):
    nested_dict = check_obj4np(nested_dict)
    json_obj = json.dumps(nested_dict, indent=4)
    with open(save_location+str(file_name)+'.json', 'a+') as f:
        f.write(json_obj)
        f.close()

    
#save_location+str(file_name)+'.csv'
def save2csv(nested_dict, file_name, save_location:str):
    
    nested_dict = check_obj4np(nested_dict)
    
    columns = list(nested_dict.keys())
    path = os.path.join(save_location, file_name +".csv")
    try:
        with open(path, "a", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=columns)
            # using dictwriter
            # using writeheader function
            if f.tell() == 0:
                writer.writeheader()
            writer.writerow(nested_dict)
            f.close()
    except IOError as e:
        print("I/O error({0}): {1}".format(e.errno, e.strerror))
    except ValueError:
              print("could not convert to string")
    except:
              print("unexpected error: ", sys.exc_info()[0])
        

def save2json(nested_dict, file_name, save_location:str):
    nested_dict = check_obj4np(nested_dict)
    #print(nested_dict)
    #print(nested_dict.items())
    json_obj = json.dumps(nested_dict, indent=4)
    #print(json_obj)
    path = os.path.join(save_location, file_name+".json")
    #print(path)
    with open(path, 'w') as f:
        f.write(json_obj)
        
        


def read_in_json(file_path, file_name):
    path = os.path.join(file_path, 'file_name')
    try:
        with open(path, 'r') as f:
            #obj = f.read()
            dj = json.load(f, object_pairs_hook= collections.OrderedDict) #obj, 
            #print(dj)
    except Exception as e:
        print("Error decoding Json")
        print(e)


class Flattern(nn.Module):
    def __init__(self):
        super(Flattern, self).__init__()
    def forward(self, x):
        #print(x.shape)
        x = x.flatten()
        return x


def choose_model(model_name, lin_lay, dropout):

    if model_name == '6c3l':
        return sixnet(in_chan=3, f_lin_lay=int(lin_lay), l_lin_lay=11, ks= (3,5), dropout= dropout)
    else:
        print('Model Name Not Recognised')




def check_model_sizes_bits(model):
    bits = 32
    mods = list(model.modules())
    sizes = []
    total_bits = 0
    
    for i in range(1,len(mods)):
        m = mods[i]
        p = list(m.parameters())
        for j in range(len(p)):
            sizes.append(np.array(p[j].size()))
    
    for i in range(len(sizes)):
        s = sizes[i]
        bitz = np.prod(np.array(s))*bits
        total_bits += bitz
    total_bytes = total_bits/8
    total_megabytes = total_bytes/1e+6
    total_gigabytes = total_megabytes/1000
    print(total_bits, 'bits    ', total_bytes, "bytes    ", total_megabytes, "MegaBytes    ", total_gigabytes,"GigaBytes") # 148480


def ptrblk_fin_mod_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    
    size_all_mb = (param_size + buffer_size) / 1024**2
    size_all_gb = size_all_mb/953.674
    print('model size: {:.3f}MB'.format(size_all_mb))
    print('model size: {:.3f}GB'.format(size_all_gb))




In [9]:

def train_val_batch(model, train, val, loop_run_name, save_dict, lr, loss_fn, epochs, batch_size, optimizer, scheduler_value, device): #train_dl, val_dl, 
    #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3) 
    #!nvidia-smi
    model.train()

    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    sample = False
    
    
    total_epochs = 0
    #!nvidia-smi
    for epoch in tqdm(range(epochs)):
        #!nvidia-smi
        if epoch == 1:
            sample = True
            random_value = random.randrange(0,batch_size)
        else:
            random_value = None
            sample = False

            #random_value = random.randrange(0,batch_size)

        print('Training...')
        #!nvidia-smi

        t_loss, train_prediction, train_targets, t_correct, model, optimizer = loop_batch(model, train, loss_fn, batch_size,sample,random_value,epoch,loop_run_name, save_dict, device, optimizer =optimizer, scheduler= None, train =True) #, scheduler =scheduler

        t_loss_list.append(t_loss)
        [t_predict_list.append(pred.argmax()) for pred in train_prediction]
        [t_label_list.append(lab.argmax()) for lab in train_targets]
        wandb.log({'t_loss':t_loss})
    
        train_acc = (t_correct/(len(train)*batch_size)*100) ###
        print('train accuracy: ', train_acc )
        t_accuracy_list.append(train_acc)
        wandb.log({'train_acc':train_acc})
        
        #print("post train and logging: ")
        #!nvidia-smi
        
        #print("After training- Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)

            
        print('validating...')
        #!nvidia-smi
        
        v_loss, val_prediction, val_targets, val_correct= loop_batch(model, val, loss_fn, batch_size,sample,random_value,epoch,loop_run_name, save_dict, device, optimizer =None, scheduler= None, train =False)

        v_loss_list.append(v_loss)
        [v_predict_list.append(pred.argmax()) for pred in val_prediction]
        wandb.log({'v_loss':v_loss})

        #print("After valdiation- Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
        #print("post validation pre logging:  ")
        #!nvidia-smi
        
        val_acc = (val_correct/(len(val)*batch_size)*100)
        v_accuracy_list.append(val_acc)
        print('validation accuracy: ', val_acc )
        wandb.log({'val_acc':val_acc})
        #print("val, post logging:  ")
        #!nvidia-smi
    
        total_epochs += 1
        
    save_dict['Current_Epoch'] = epochs
    save_dict['training_samples'] = len(train)
    save_dict['validation_samples'] = len(val)
    
    save_dict['t_accuracy_list'] = t_accuracy_list 
    save_dict['v_accuracy_list'] = v_accuracy_list  #
            
    #model = best_model
    save_dict['t_loss_list'] = t_loss_list
    save_dict['v_loss_list'] = v_loss_list
    
    save_dict['t_labels'] = train_targets
    save_dict['v_labels'] = val_targets
    
    save_dict['t_predict_list'] = t_predict_list 
    save_dict['v_predict_list'] = v_predict_list  #
    
    return model, save_dict

from functions import ImageProcessor

def loop_batch(model, data, loss_fn, batch_size, sample,random_value,epoch,loop_run_name, save_dict, device, optimizer, scheduler= None, train =True):	# Train and Val loops. Default is train
    #model = model
    #print("loop batch start:  ")
    #!nvidia-smi
    
    total_samples = len(data)
    if train:
        model.train()
        where ='tra'
        #lr_ls = []
    else:
        model.eval()   #  (torch.Size([16, 11])) that is different to the input size (torch.Size([11]))
        where = 'val'
    #print("loop start- Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
    #print("set model to train or eval")
    #!nvidia-smi

    predict_list = []
    total_count = 0
    num_correct = 0
    current_loss = 0
    labels =[]

    
    for i, batch in enumerate(data,0):
        
        #print("starting looping:  ")
        #!nvidia-smi
        
        x_batch, y_batch = batch
        if sample == True:
            IP = ImageProcessor(device) #img, scale:int, save_dict;dict, epoch:int, where:str
            i = IP.view(x_batch[random_value],1, loop_run_name, save_dict, epoch, where)
            # 
            sample= False

        prediction = model.forward(x_batch)

        loss = loss_fn(prediction, y_batch)
        #print("got prediction and loss:  ")
        #!nvidia-smi
        
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        #print(" optimizer loss activated:  ")
        #!nvidia-smi
        
        [predict_list.append(pred.argmax().to("cpu")) for pred in prediction]#.argmax())
        [labels.append(y.argmax().to("cpu")) for y in y_batch]
        #print("added pred and labels to lists")
        #!nvidia-smi
        
        for i in range(len(y_batch)-1):
            if y_batch[i].argmax() == prediction[i].argmax():
                num_correct +=1
        del y_batch
        del prediction
        
        #print("deleted var pred and label:  ")
        #!nvidia-smi

        total_count+= batch_size
        current_loss += loss.item()
        del loss

    if train:
        return current_loss, predict_list, labels, num_correct, model, optimizer #, lr_ls
    else:
        return current_loss, predict_list, labels, num_correct


## model, data, loss_fn, device, optimizer =None, scheduler= None, train =True
def test_loop_batch(model,data, loss_fn, batch_size, device):
    model = model.eval()
    predict_list = []
    label_list = []
    total_count =0
    num_correct = 0
    correct = 0
    

    with torch.no_grad():
        for i, batch in enumerate(data,0):
            #tense = tense.to(device)
            tense, label = batch
            #label = label.to(device)
            
            prediction = model.forward(tense) #.to(device)
            #print('p', prediction.shape, 'l ', label.shape)
            #label = label_oh_tf(Y[idx], device, num_classes)
            for i in range(len(label)-1):
                #print(len(label), label[0].argmax(), len(label)-1)
                if label[i].argmax() == prediction[i].argmax():
                    num_correct +=1
            [predict_list.append(pred.argmax().to("cpu")) for pred in prediction]
            [label_list.append(lab.argmax().to("cpu")) for lab in label]
            
            total_count += batch_size

            del prediction
            del label

        acc = num_correct/total_count
        accuracy = 100*(acc)

        print(accuracy)
        return accuracy, predict_list, label_list



def get_data(random_seed):
    file_path =  data_path
    #print(file_path)
    img_len = len(os.listdir(file_path))
    
    x, y = import_imagedata(file_path)
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, train_size=0.7,
                                     random_state=random_seed, shuffle=True)
    x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.3, train_size=0.7,
                                     random_state=random_seed, shuffle=True)

    return x_train, y_train, x_val, y_val, x_test, y_test


    
def get_lin_lay(model_card, resolution):
    if resolution == [452, 144]:
        lin_lay = model_card['f_lin_lay'][0]
    elif resolution == [226, 72]:
        lin_lay = model_card['f_lin_lay'][1]
    elif resolution == [113, 36]:
        lin_lay = model_card['f_lin_lay'][2]
    elif resolution == [57, 18]:
        lin_lay = model_card['f_lin_lay'][3]
    elif resolution == [29, 9]:
        lin_lay = model_card['f_lin_lay'][4]
    elif resolution == [15, 5]:
        lin_lay = model_card['f_lin_lay'][5]
    elif resolution == [8, 3]:
        lin_lay = model_card['f_lin_lay'][6]
    else:
        print("PARAMETER NOT FOUND: \n f_lin_lay FROM MODEL CARD")
    return lin_lay


In [10]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"


def _go(config=None):
    diction = {}
    
    #print('1')
    #!nvidia-smi
    
    #print("Max allocated memory (GB):", torch.cuda.max_memory_allocated() / 1024 ** 3)
    
    if len(gitHASH) <1:
        print("YOU FORGET THE GIT HASH")
        return
    else:
        #print('Git Hash registered')
        pass
        
    with wandb.init(config=config, project=f"Big Loop batching of model 6c3l", notes="big loop batcing 6c3l.  060924",):
        config = wandb.config
        start = time.process_time()
        best_acc = 0
            
        for model_idx, model_card in enumerate(config['model_cards']):
            #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                    
            model_name = model_card['model']
            model_index = model_card['idx']
            dropout = model_card['dropout'] 
            for res_idx, resolution_card in enumerate(config['resolution_cards']):
                #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
            
                resolution = resolution_card['resolution']
                pad = resolution_card['padding']
                lin_lay = get_lin_lay(model_card, resolution)
                print('lin lay', lin_lay)
            
                for lr_idx, lr in enumerate(config['learning_rate_cards']):
                    for wd_idx, wd_card in enumerate(wd_cards):
                        for seed_idx, seed in enumerate(config['seeds']):
                            seed = seed
                            for lossfn_idx, loss in enumerate(config['loss_fn_cards']):
                                
                                #torch.cuda.empty_cache()
                                #print('2')
                                #!nvidia-smi

                                config['batch_size']

                                print('Model: ', str(model_name), f" idx: {model_idx} / {len(config.model_cards)}")
                                print('resolution: ', str(resolution), f" idx: {res_idx} / {len(config['resolution_cards'])}")
                                print('learning rate: ', str(lr), f" idx: {lr_idx} / {len(config['learning_rate_cards'])}")
                                print('weight decay: ', str(wd_card), f" idx: {wd_idx} / {len(config['wd_cards'])}")
                                #print('scheduler: ', str(scheduler_value), f" idx: {sched_idx} / {len(config['scheduler_cards'])}")
                                print('seed: ', str(seed), f" idx: {seed_idx} / {len(config['seeds'])}")
                                print('loss function: ', str(loss), f" idx: {lossfn_idx} / {len(config['loss_fn_cards'])}")
                                print('Batch size: ', config['batch_size'])
                                print('Training epochs: ', config['epochs'])
                                run_start_time = time.process_time()
                                print('start time: ',run_start_time)

                                print(time.process_time() - start)

                                epochs = config['epochs'] #40

                                IP = ImageProcessor(device)

                                wandb.log({'gitHash':gitHASH})
                                wandb.log({'Epochs': epochs})
                                
                                #print('3')
                                #!nvidia-smi
                                
                                # set save dictionary
                                save_dict = {'Run' : f"{model_name}_{resolution}_{date}",
                                             'Current_Epoch': 0,
                                             'save_location' : _save_location}
      
                                model = choose_model(model_name, lin_lay, dropout).to(device)
                                #print("Before model init - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                                #model = smallnet3(in_chan=3, f_lin_lay=int(lin_lay), l_lin_lay=11, ks= (3,5), dropout= dropout).to(device)

                                #print('4')
                                #!nvidia-smi
                                best_model = deepcopy(model)

                                #print("After model init, Before data loading - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)

                                x_train, y_train, x_val, y_val, x_test, y_test = get_data(seed)
                                av_lum = IP.new_luminance(x_train)
                                #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                                
                                train_ds = IDSWDataSetLoader2(x_train, y_train, resolution,pad,av_lum,model_name, device)# av_lum, res,pad,
                                train = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2

                                
                                test_ds = IDSWDataSetLoader2(x_test, y_test, resolution,pad,av_lum,model_name, device)
                                test = DataLoader(test_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2
                                #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                                val_ds = IDSWDataSetLoader2(x_val, y_val, resolution,pad,av_lum,model_name, device)
                                val = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2
                                
                                #print("After data loading - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)


                                loss_fn = set_lossfn(loss)
                                
                                # set optimizer
                                optimizer = torch.optim.Adam(model.parameters(),lr=lr)

                                wandb.watch(model, loss_fn, log='all', log_freq=2, idx = model_index)
                                #print('6')
                                #!nvidia-smi
                                loop_run_name = f"{save_dict['Run']}_{resolution}_{lr}_0_{seed}_{loss}"
     
                                model, save_dict=  train_val_batch(model, train,val, loop_run_name,save_dict, lr, loss_fn,epochs, config['batch_size'], optimizer, 0, device)

                                

                                test_acc,test_predict_list, y_test = test_loop_batch(model,test, loss_fn, config['batch_size'], device) #model, model_name, X, Y, res, pad, loss_fn, device, num_classes=11
                                
                                #print(test_predict_list)
                                print(' \n train Acc: ', save_dict['t_accuracy_list'][-1])
                                print(' \n val Acc: ', save_dict['v_accuracy_list'][-1])
                                print(' \n test Acc: ', test_acc)

                                save_dict.update({'test_acc': test_acc})
                                save_dict.update({'test_predict': test_predict_list})
                                save_dict.update({'test_labels': list(y_test)})

                                learning_curve(save_dict['t_loss_list'], save_dict['v_loss_list'], save_location=save_dict['save_location'],run_name=loop_run_name)
                                accuracy_curve(save_dict['t_accuracy_list'], save_dict['v_accuracy_list'],save_location=save_dict['save_location'],run_name=loop_run_name)
                                test_predict_list=[pred.cpu() for pred in test_predict_list]
                                plot_confusion(predictions= test_predict_list, actual= y_test, title = "Test Confusion matrix", run_name = loop_run_name,save_location =save_dict['save_location'])
                                
                                wandb.log({'test_acc': test_acc})
                                wandb.log({'test_predict': test_predict_list})
                                wandb.log({'test_labels': list(y_test)})
                                d = date.today()
                                d = str(d)

                                save_dict.update({'Date':d})
                                save_dict.update({'gitHASH':str(gitHASH)})
                                save_dict.update({'model_name': str(model_name)})
                                save_dict.update({'loss_fn': str(loss)})
                                save_dict.update({'lr': str(lr)})
                                save_dict.update({'wd': str(wd_card)})
                                
                                save_dict.update({'seed': str(seed)})
                                save_dict.update({'resolution': str(resolution)})
                                save_dict.update({'pad': int(pad)})
                                save_dict.update({'lin_lay': int(lin_lay)})

                                
                                save_dict.update({'run time': (time.process_time() - run_start_time)})
                                #save_dict.update({'test_loss':test_loss})
                                
                                title = save_dict['Run']
                                save2json(save_dict, loop_run_name, _save_location)
                                
                                save2csv(save_dict, title, _save_location)
                                #saving
                                if save_dict['v_accuracy_list'][-1] > best_acc:
                                    best_acc = save_dict['v_accuracy_list'][-1]
                                    best_model = deepcopy(model)#.to(device)
                                    diction.update(save_dict)
                                    diction['model_state_dict'] = best_model.state_dict()
                                
                                
                                clear_output()
                                
                                print(f' \n END {model_name} {resolution} Run Time: ',time.process_time() - run_start_time)
                                #!nvidia-smi
                                torch.cuda.empty_cache()

        
        with open(f"{_save_location}{loop_run_name}.pt", 'wb+') as f:
           torch.save(diction, f)
        with open(f"{_save_location}{loop_run_name}.pkl", 'wb+') as f:
           torch.save(diction, f)
        print('Final Run time: ',time.process_time() - start)

In [11]:
_go(config)

 
 END 6c3l [8, 3] Run Time:  195.86698432000003
Final Run time:  864.860400803


Epochs,▁▁▁▁▁▁▁
t_loss,█▆▁█▆▂██▅████████████
test_acc,██▆▁▂▁▂
train_acc,▂▄█▂▅█▁▃▇▂▂▂▁▁▂▂▂▂▁▂▂
v_loss,█▅▁█▄▁██▃████████████
val_acc,▅▆█▂▅█▃▅█▂▂▂▁▂▃▂▂▂▂▁▁
Epochs,3
gitHash,f7ff2f83eded46c54339...
t_loss,2.14789
test_acc,10.54688
train_acc,9.85577


In [12]:
# 12.58 GiB. GPU 0 has a total capacty of 23.65 GiB of which 8.04 GiB is free
23.65-8.04

15.61


pred torch.Size([11])

lab  torch.Size([5, 11])

060924
schedulaer values replaced with 0 to reduce varaibles in memory.